### Import necessary libraries

In [1]:
import os
import io
import cv2
import sys
import timm
import h5py
import torch
import joblib
import random
import numpy as np
import pandas as pd
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from typing import Optional,Callable,Any
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
from tqdm.auto import tqdm

sys.path.append('../..')
from src.utils import load_env_vars


In [2]:
import warnings
warnings.filterwarnings("ignore")

### Device for inference

In [3]:
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using {DEVICE} for inference')

Using cuda for inference


### Reproducibility

In [4]:
def seed_everything(seed : int) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

### Global variables

In [23]:
DATA_DIR, _, _ = load_env_vars()

In [24]:
INPUT_DIR = '/kaggle/input/'
WORKING_DIR = '/kaggle/working/'
EVA_MODELS = '/kaggle/input/eva02/pytorch/eva-rgb-336/3'
COMP_DIR = os.path.join(INPUT_DIR, 'isic-2024-challenge')

In [25]:
MODEL_NAME = 'eva02_small_patch14_336.mim_in22k_ft_in1k'
IMAGE_SIZE = 336
BATCH_SIZE = 64
NUM_CLASSES = 1
IN_CHANNELS = 3

In [26]:
SEED = 42
seed_everything(SEED)

### Utils

In [27]:
def remove_black_border(image):

    mask = image < 20
    mask = np.all(mask, axis=2)
    coords = np.argwhere(~mask)
    x0, y0 = coords.min(axis=0)
    x1, y1 = coords.max(axis=0)
    cropped = image[x0:x1, y0:y1]

    return cv2.resize(cropped, (image.shape[1], image.shape[0]))

### Data loading

In [28]:
train_meta_path = os.path.join(DATA_DIR, 'isic_2024/train-metadata.csv')
train_images_path = os.path.join(DATA_DIR, 'isic_2024/train-image.hdf5')

In [29]:
train_metadata = pd.read_csv(train_meta_path)

### ISIC Dataset

In [30]:
class ISICDataset(Dataset):
    def __init__(self, df, hdf5_file, transforms=None):
        self.hdf5 = h5py.File(hdf5_file, mode='r')
        self.df = df
        self.ids = self.df['isic_id'].values
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        id = self.ids[idx]
        img = np.array(Image.open(io.BytesIO(self.hdf5[id][()])))

        img = remove_black_border(img)

        if self.transforms:
            img = self.transforms(image=img)['image']

        return {'image': img}

### EVA02 Architecture

In [31]:
class ISICModel(nn.Module):
    def __init__(self, model_name, in_channels=3, num_classes=1, pretrained=True, checkpoint=None):
        super(ISICModel, self).__init__()

        # create model
        self.model = timm.create_model(model_name, pretrained=pretrained, in_chans=in_channels, checkpoint_path=checkpoint)

        # get in_features
        in_features = self.model.head.in_features

        # set classifier
        self.model.head = nn.Linear(in_features, num_classes)

        # sigmoid function
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):

        # forward pass
        y = self.model(x)

        # apply sigmoid
        y = self.sigmoid(y)

        return y

### Define Dataloader

In [32]:
def create_transforms(image_size : int) -> Callable:

    transforms = A.Compose([
        A.Resize(image_size, image_size),
        A.Normalize(mean=[0.485, 0.456, 0.406], 
                    std=[0.229, 0.224, 0.225], 
                    max_pixel_value=255.0, 
                    p=1.0),
        ToTensorV2()
    ], p=1.0)
    
    return transforms

In [33]:
transforms = create_transforms(image_size=IMAGE_SIZE)

dataset = ISICDataset(df=train_metadata, hdf5_file=train_images_path, transforms=transforms)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

### Inference

In [34]:
def inference(model, dataloader, device):
    model.to(device)
    model.eval()
    predictions = []

    with torch.no_grad():
        for data in tqdm(dataloader):
            images = data['image'].to(device, dtype=torch.float)
            outputs = model(images)
            predictions.extend(outputs.detach().cpu().numpy())

    return predictions

In [35]:
eva_models_files = ['eva1.pth', 'eva2.pth', 'eva3.pth', 'eva4.pth', 'eva5.pth']
eva_models = []
eva_preds = []
EVA_MODELS = '/home/bracs/Desktop/models'

for model_file in eva_models_files:
    model = ISICModel(model_name=MODEL_NAME, in_channels=3, num_classes=1, pretrained=False)
    model.load_state_dict(torch.load(os.path.join(EVA_MODELS, model_file)))
    model_predictions = inference(model, dataloader, DEVICE)
    eva_models.append(model)
    eva_preds.append(model_predictions)

  0%|          | 0/6267 [00:00<?, ?it/s]

In [ ]:
eva_preds = np.array(eva_preds)
eva_preds_mean = np.mean(eva_preds, axis=0)

### Save new dataframe

In [19]:
train_eva02 = pd.DataFrame(columns=['isic_id', 'eva02_preds'])
train_eva02['isic_id'] = train_metadata['isic_id']
train_eva02['eva02_preds'] = eva_preds_mean

In [21]:
train_eva02[['isic_id', 'eva02_preds']].to_csv('train_eva02_v2.csv', index=False)
train_eva02.head()

,isic_id,eva02_preds
0,ISIC_0015670,0.006783
1,ISIC_0015845,0.109842
2,ISIC_0015864,0.001834
3,ISIC_0015902,0.028074
4,ISIC_0024200,0.017649
